<a href="https://colab.research.google.com/github/chaupmbn/Image_Retrieval/blob/master/Flickr_Image_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install selenium

!apt-get update
!apt-get install -y wget
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,173 kB]
Get:11 https://ppa.launchpadcontent.net

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup # For parsing HTML content
from urllib.parse import urljoin, urlparse # For handling URLs
import urllib.request # For making HTTP requests
import time # For handling time-related operations
import os # For interacting with the operating system (relate to dir, folder, file)
from tqdm import tqdm # For displaying progress bars (visualize progress)
import concurrent.futures # For multi-threading
import json # For writing to a text file
from PIL import Image # For handling images

In [6]:
class UrlScraper:
    # Constructor
    def __init__(self, url_template, max_images=50, max_workers=4):
        self.url_template = url_template # Link crawl
        self.max_images = max_images # Max images
        self.max_workers = max_workers # Thread
        self.setup_environment() # Call for set up environment

    # Set up environment for selenium
    def setup_environment(self):
        os.environ['PATH'] += ':/usr/lib/chromium-browser/'
        os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

    def get_url_images(self, term):
        """
        Crawl the urls of images by term

        Parameters:
        term (str): The name of animal, plant, scenery, furniture

        Returns:
        urls (list): List of urls of images
        """

        # Initialize Chrome driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)

        url = self.url_template.format(search_term=term)
        driver.get(url)

        # Start crawl urls of image like brute force - the same mechanism with
        # this but add some feature
        urls = []
        more_content_available = True

        pbar = tqdm(total=self.max_images, desc=f"Fetching images for {term}")
        # Set up for visualize progress

        while len(urls) < self.max_images and more_content_available:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            img_tags = soup.find_all("img")

            for img in img_tags:
                if len(urls) >= self.max_images:
                  break
                if 'src' in img.attrs:
                    href = img.attrs['src']
                    img_path = urljoin(url, href)
                    img_path = img_path.replace("_m.jpg", "_b.jpg").replace(
                        "_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                    if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                        continue
                    urls.append(img_path)
                    pbar.update(1)

            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH,
                    '//button[@id="yui_3_16_0_1_1721642285931_28620"]')))
                load_more_button.click()
                time.sleep(2)
            except:
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

                new_soup = BeautifulSoup(driver.page_source, "html.parser")
                new_img_tags = new_soup.find_all("img", loading_="lazy")
                if len(new_img_tags) == len(img_tags):
                    more_content_available = False
                    img_tags = new_img_tags

        pbar.close()
        driver.quit()
        return urls

    def scrape_urls(self, categories):
        """
        Call get_url_images method to get all urls of any object in categories\

        Parameter:
        categories (dictionary): the dict of all object we need to collect image
            with format categories{"name_object": [value1, value2, ...]}

        Returns:
        all_urls (dictionary): Dictionary of urls of images
        """
        all_urls = {category: {} for category in categories}

        # Handle multi-threading for efficent installation
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                for category, terms in categories.items() for term in terms}

            for future in tqdm(concurrent.futures.as_completed(future_to_term),
                          total=len(future_to_term), desc="Overall Progress"):
                category, term = future_to_term[future]
                try:
                    urls = future.result()
                    all_urls[category][term] = urls
                    print(f"\nNumber of images retrieved for {term}: {len(urls)}")
                except Exception as exc:
                    print(f"\n{term} generated an exception: {exc}")
        return all_urls

    def save_to_file(self, data, filename):
        """
        Save the data to a JSON file.

        Parameters:
        data (dict): The data to be saved.
        filename (str): The name of the JSON file.

        Returns:
        None
        """
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {filename}")

In [7]:
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox",
               "Civet", "Pangolins", "Rabbit", "Bats", "Whale", "Cock", "Owl",
               "flamingo", "Lizard", "Turtle", "Snake", "Frog", "Fish",
               "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly",
               "Flies", "Mosquito", "Ants", "Cockroaches", "Spider",
               "scorpion", "tiger", "bird", "horse", "pig", "Alligator",
               "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel",
               "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck",
               "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower",
              "Flower", "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss",
              "Grass", "Palmtree", "Corn", "Tulip", "Rose", "Clove",
              "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani",
              "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree",
              "RangoonCreeper", "Jackfruit", "Cottonplant", "Corneliantree",
              "Coffeeplant", "Coconut", "wheat", "watermelon", "radish",
              "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks",
                  "table", "Piano", "Bookcase", "Umbrella", "Clothes", "cart",
                  "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies",
                "Lake", "desert", "farmland", "river", "hedges", "plain",
                "sky", "cave", "cloud", "flowergarden", "glacier", "grassland",
                "horizon", "lighthouse", "plateau", "savannah", "valley",
                "volcano", "waterfall"]
}
urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"],
                     max_images=20, max_workers=5)
image_urls = scraper.scrape_urls(categories)
scraper.save_to_file(image_urls, 'image_urls.json')

Fetching images for Dog:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Cat:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for cows:   0%|          | 0/20 [00:00<?, ?it/s]



Fetching images for Elephant:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Cat:   5%|▌         | 1/20 [00:04<01:31,  4.82s/it]

Fetching images for Dog:   5%|▌         | 1/20 [00:05<01:43,  5.43s/it]


Fetching images for cows:   5%|▌         | 1/20 [00:05<01:38,  5.20s/it]



Overall Progress:   1%|          | 1/134 [01:31<3:22:29, 91.35s/it]


Number of images retrieved for Cat: 20


Fetching images for Elephant: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]



Number of images retrieved for Dog: 20


Overall Progress:   2%|▏         | 3/134 [01:33<46:13, 21.17s/it]  

...


Number of images retrieved for valley: 20


Overall Progress:  99%|█████████▉| 133/134 [21:54<00:07,  7.03s/it]


Number of images retrieved for volcano: 20


Overall Progress: 100%|██████████| 134/134 [21:56<00:00,  9.82s/it]


Number of images retrieved for waterfall: 20
Data saved to image_urls.json


In [9]:
class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset',
                 max_workers=4, delay=1):
        self.json_file = json_file
            # file containing URLs of images in JSON format
        self.download_dir = download_dir # Folder name for storing images
        self.max_workers = max_workers # Number of threads
        self.delay = delay
            # Polite delay: when we send request too much to the server for
            # downloading images without polite delay,
            # it will crash or prevent your IP from being blocked

        self.filename = set() # To store filename directories
        self.setup_directory() # Set up the folder structure

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
          os.makedirs(self.download_dir)

    def read_json(self):
        """
        Read the JSON file and return the data.

        Returns:
        data (dict): The data read from the JSON file.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Check if the URL is valid.

        Parameters:
        url (str): The URL to be checked.

        Returns:
        bool: True if the URL is valid, False otherwise.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False
    def download_image(self, url, category, term, pbar):
        """
        Download the image from the given URL.

        Parameters:
        url (str): The URL of the image to be downloaded.
        category (str): The category of the image.
        term (str): The term or keyword associated with the image.
        pbar (tqdm): The progress bar object.

        Returns:
        str: A message indicating the status of the download.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))

        self.filename.add(filename) # Record the filename directory

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
        return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Download images from the URLs in the JSON file.

        Returns:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values()
                            for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(
                max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(
                                self.download_image, url, category, term, pbar))

                            time.sleep(self.delay) # Polite delay

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Export the filename directories to a text file.

        Returns:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")

In [10]:
downloader = ImageDownloader(json_file='image_urls.json',
                             download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()
downloader.export_filename()

Downloaded: https://live.staticflickr.com/169/384447781_c811dbf520_b.jpg
Downloaded: https://live.staticflickr.com/10/13350698_3eee621f07_b.jpg
Downloaded: https://live.staticflickr.com/7102/7380287404_ccf55f560d_b.jpg
Downloaded: https://live.staticflickr.com/65535/51130913602_fa56f5b380_b.jpg
Downloaded: https://live.staticflickr.com/2027/2534782254_cbc0e12bd4_b.jpg
Downloaded: https://live.staticflickr.com/3225/3018520259_7b9fdf784a_b.jpg
Downloaded: https://live.staticflickr.com/5054/5451164696_92de322f65_b.jpg
Downloaded: https://live.staticflickr.com/6169/6164665781_594df941fb_b.jpg
Downloaded: https://live.staticflickr.com/3495/3871029191_94c00c8220_b.jpg
Downloaded: https://live.staticflickr.com/3101/2374292304_809e4f6803_b.jpg
Downloaded: https://live.staticflickr.com/8464/8363470909_a3f7515f72_b.jpg
Downloaded: https://live.staticflickr.com/4133/5044088454_47e919a6f9_b.jpg
Downloaded: https://live.staticflickr.com/8315/27882103054_9d980562e7_b.jpg
Downloaded: https://live.sta

Downloaded: https://live.staticflickr.com/65535/50053838501_2ab958a8b0_b.jpg
Downloaded: https://live.staticflickr.com/7062/6836614904_5dc2767621_b.jpg
Downloaded: https://live.staticflickr.com/8502/8258264811_1cb016efe8_b.jpg
Downloaded: https://live.staticflickr.com/3576/3324696288_8f0b635475_b.jpg
Downloaded: https://live.staticflickr.com/7837/46461302315_11cfd23574_b.jpg
Downloaded: https://live.staticflickr.com/8251/8470846267_0a583b8c44_b.jpg
Downloaded: https://live.staticflickr.com/8252/8470843157_0e0ec78433_b.jpg
Downloaded: https://live.staticflickr.com/3856/33375915312_b32633a75d_b.jpg
Downloaded: https://live.staticflickr.com/6027/6097411524_e7365af0ae_b.jpg
Downloaded: https://live.staticflickr.com/2292/2377602381_2944351e93_b.jpg
Downloaded: https://live.staticflickr.com/3261/2378439644_8b6585c21a_b.jpg
Downloaded: https://live.staticflickr.com/7381/27618826684_fe0f484253_b.jpg
Downloaded: https://live.staticflickr.com/7466/15639965814_d913d2d209_b.jpg
Downloaded: https:/

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
def check_and_preprocess_images(image_dir):
    """
    Check and preprocess images in the specified directory.

    Parameters:
    image_dir (str): The directory containing the images to be checked and preprocessed.

    Returns:
    None
    """
    for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Check if image is smaller than 50x50 pixels
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Convert non-RGB images to RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # If file is not an image, delete it
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

check_and_preprocess_images('Dataset')

Converted Dataset/scenery/desert/48115750033_ed68297f86_b.jpg to RGB
Converted Dataset/scenery/desert/49007511591_74582e367b_b.jpg to RGB
Converted Dataset/scenery/desert/52712003999_cba6904a86_b.jpg to RGB
Converted Dataset/scenery/desert/49011833281_72063e1544_b.jpg to RGB
Converted Dataset/scenery/volcano/49629211101_d46f735055_b.jpg to RGB
Converted Dataset/animal/Elephant/4839032364_8c521066b2_b.jpg to RGB
Converted Dataset/animal/Cheetah/4838391253_bed2279fdf_b.jpg to RGB
Converted Dataset/animal/Cheetah/4838389045_d5fc08ca2b_b.jpg to RGB
Converted Dataset/plant/Bean/8529951413_c315df19e5_b.jpg to RGB
Converted Dataset/plant/Wildflower/51418402341_b87250d90c_b.jpg to RGB


In [14]:
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/furniture/ (stored 0%)
  adding: Dataset/furniture/chair/ (stored 0%)
  adding: Dataset/furniture/chair/6966550587_4760de012a_b.jpg (deflated 3%)
  adding: Dataset/furniture/chair/31972477316_88e1304de3_b.jpg (deflated 3%)
  adding: Dataset/furniture/chair/7169739223_4c57d346c3_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/8288177868_2cd17a8fb5_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/2433092503_fe6cf1e688_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/2475070939_7b7338b4d8_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/177715520_deee6ea440_b.jpg (deflated 1%)
  adding: Dataset/furniture/chair/147482645_d125c7f1f8_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/2865338391_a419230b3e_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/3619633442_0c55846789_b.jpg (deflated 1%)
  adding: Dataset/furniture/chair/2556879813_fa775d223a_b.jpg (deflated 0%)
  adding: Dataset/furniture/chair/5894919073_f

In [22]:
import os
import shutil
from collections import defaultdict

# Define the source and target directories
source_dir = "Dataset"
train_dir = "data/train"
test_dir = "data/test"

# Create the target directories if they don’t exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Initialize a dictionary to hold file paths for each class
class_files = defaultdict(list)

# Read the file paths from the text file
with open('filename.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line:
            # Extract the class name from the path
            parts = line.split('/')
            class_name = parts[2] # Structure Dataset/category/class/image.jpg
            class_files[class_name].append(line)
class_files

# Move images to the train and test directories
for class_name, files in class_files.items():
    # Create the train and test directories for the class
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move 19 images to train and 1 image to test
    for i, file_path in enumerate(files):
        if i == 0:
            shutil.copy(file_path, test_class_dir)
        elif i < 20:
            shutil.copy(file_path, train_class_dir)

print("Dataset organization complete!")

Dataset organization complete!


In [23]:
!zip -r /content/drive/MyDrive/data.zip data

updating: data/ (stored 0%)
updating: data/train/ (stored 0%)
updating: data/train/waterfall/ (stored 0%)
updating: data/test/ (stored 0%)
updating: data/test/waterfall/ (stored 0%)
updating: data/test/waterfall/9536683732_3c101945da_b.jpg (deflated 0%)
  adding: data/train/Mushroom/ (stored 0%)
  adding: data/train/Mushroom/4961199453_557599aa0b_b.jpg (deflated 1%)
  adding: data/train/Mushroom/481012156_f8751644b8_b.jpg (deflated 0%)
  adding: data/train/Mushroom/30773281961_a2983bf1be_b.jpg (deflated 1%)
  adding: data/train/Mushroom/29359074785_d9567082ce_b.jpg (deflated 0%)
  adding: data/train/Mushroom/3397647648_48b7c31722_b.jpg (deflated 0%)
  adding: data/train/Mushroom/21545048500_e15da413a7_b.jpg (deflated 0%)
  adding: data/train/Mushroom/189098954_f360f4bdfe_b.jpg (deflated 0%)
  adding: data/train/Mushroom/3607099094_eed0cd0fed_b.jpg (deflated 0%)
  adding: data/train/Mushroom/846207623_72126d2f02_b.jpg (deflated 0%)
  adding: data/train/Mushroom/52954574_058d5c59b3_b.jpg